In [2]:
# -*- coding: utf-8 -*-
import pymongo as mg
import datetime
import plotly
from datetime import datetime, timedelta

%config IPCompleter.greedy=True

#ECHO ptt Start ====================================
fetchdays = 1
yesterday_temp = datetime.now() - timedelta(days = fetchdays)
yesterday_date = yesterday_temp.strftime("%Y-%m-%d")
today_date = datetime.now().strftime("%Y-%m-%d")
ary1 = str(yesterday_date).split("-")
start = datetime(int(ary1[0]),int(ary1[1]),int(ary1[2]), 0, 0, 0)
ary1 = str(today_date).split("-")
end =  datetime(int(ary1[0]),int(ary1[1]),int(ary1[2]), 23, 59, 59)

start = datetime(2020, 10, 8, 0, 0, 0)
end = datetime(2020, 11, 15, 23, 59, 59)
client = mg.MongoClient('127.0.0.1:27017')
db = client["ptt"]
col = db['Gossiping']
sort = [("date", -1)]
result = col.find({"date":{'$gte': start, '$lte': end}})
result  =  list(result)

temp_dict = {}
date_dict = {}
for x in result:
    temp_dict[ x['url'] ] = x

print ("distinct %s records found." % (len(temp_dict) ) )

post_dict = {}
post_counter = 0
last_sum = 0
for x in temp_dict:
    last_sum = 0
    post_counter = 0
    #dt = temp_dict[x]["date"].strftime("%Y-%m-%d")
    dt = temp_dict[x]["date"].strftime("%Y-%m-%d:%H")
    #dt = temp_dict[x]["date"].strftime("%Y-%m-%d:%H:%M:%S")
    try:
        last_sum = post_dict[dt]['counter']
        post_dict[dt]={'counter':last_sum+1}
    except Exception as e:
        post_counter = post_counter+1
        post_dict[dt]={'counter':post_counter}
        pass

x_axis = []
y_axis = []
for x in post_dict:
    x_axis.append(x)
    y_axis.append(post_dict[x]['counter'])

zipped = zip(x_axis, y_axis)
x_axis = []
y_axis = []
zipped = sorted(zipped, key=lambda x: x[0])
for x in zipped:
    x_axis.append(x[0])
    y_axis.append(x[1])
print(x_axis)
print(y_axis)

distinct 58 records found.
['2020-10-11:00', '2020-10-11:15', '2020-10-21:12', '2020-10-23:00', '2020-10-24:01', '2020-10-24:02']
[1, 1, 1, 1, 42, 12]


In [3]:
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({
        "data": [{
                "x": x_axis,
                "y": y_axis
            }],
        "layout": {
            "title": "David PTT 八卦版每日發文數統計"
        }
    })

In [4]:
tmp_score = 0
lastSumScore = 0
for x in temp_dict:
    #dt = temp_dict[x]["date"].strftime("%Y-%m-%d")
    dt = temp_dict[x]["date"].strftime("%Y-%m-%d:%H")
    #print(dt)
    try:
        #print(temp_dict[x]['comments'])
        score = 0
        for y in list(temp_dict[x]['comments']):
            try:
                lastSumScore = date_dict[dt]['score']
                score = lastSumScore+y['score']
                date_dict[dt]={'date':dt,'score':score}
            except Exception as e:
                score = y['score']
                date_dict[dt]={'date':dt,'score':score}
                pass
    except Exception as e:
        #print('Exception:'+str(e))
        pass
#print(date_dict)
x_axis = []
y_axis = []
for x in date_dict:
    x_axis.append(x)
    y_axis.append(date_dict[x]['score'])

zipped = zip(x_axis, y_axis)
x_axis = []
y_axis = []
zipped = sorted(zipped, key=lambda x: x[0])
for x in zipped:
    x_axis.append(x[0])
    y_axis.append(x[1])
print(x_axis)
print(y_axis)

['2020-10-11:00', '2020-10-11:15', '2020-10-21:12', '2020-10-23:00', '2020-10-24:01', '2020-10-24:02']
[-1, 46, 9, 30, 191, 1]


In [5]:
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({
        "data": [{
                "x": x_axis,
                "y": y_axis
            }],
        "layout": {
            "title": "David PTT 八卦版每日互動(推-噓)觀察"
        }
    })
